In [ ]:
!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

from ultralytics import YOLO

from IPython.display import display, Image

import cv2
import os
import zipfile
import torch
import numpy as np
from google.colab.patches import cv2_imshow

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 30.0/78.2 GB disk)


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
!unzip /content/drive/MyDrive/data1.zip -d ./

Archive:  /content/drive/MyDrive/data1.zip
   creating: ./data1/
  inflating: ./data1/001.mp4         
  inflating: ./data1/002.mp4         
  inflating: ./data1/003.mp4         
  inflating: ./data1/004.mp4         
  inflating: ./data1/005.mp4         
  inflating: ./data1/006.mp4         
  inflating: ./data1/007.mp4         
  inflating: ./data1/008.mp4         
  inflating: ./data1/009.mp4         
  inflating: ./data1/010.mp4         
  inflating: ./data1/011.mp4         
  inflating: ./data1/012.mp4         
  inflating: ./data1/013.mp4         
  inflating: ./data1/014.mp4         
  inflating: ./data1/015.mp4         
  inflating: ./data1/016.mp4         
  inflating: ./data1/017.mp4         
  inflating: ./data1/018.mp4         
  inflating: ./data1/019.mp4         
  inflating: ./data1/020.mp4         
  inflating: ./data1/021.mp4         
  inflating: ./data1/022.mp4         
  inflating: ./data1/023.mp4         
  inflating: ./data1/024.mp4         
  inflating: ./data1/02

In [ ]:
pip install easyocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 21.6 MB/s eta 0:00:00


In [ ]:
import easyocr
reader = easyocr.Reader(['en'])

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [ ]:
# Load model from local
Model_Name = "./best.pt"
Best_Model = YOLO(Model_Name)

In [ ]:
def extract_frames(input_video_path, output_video_path):
    '''
    Shrink Video size by 2 to improve compute time
    '''

    # Open the input video
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Extract every second frame
        if frame_count % 10 == 0:
            out.write(frame)

        frame_count += 1

    # Release everything when the job is finished
    cap.release()
    out.release()

    print(f"New video created: {output_video_path}")

In [ ]:
def list_mp4_files(directory):
    '''
    Obtain file name from directory
    ret : .mp4 file names (list)
    '''
    mp4_files = []

    video_basename = []

    for file in os.listdir(directory):
        if file.endswith(".mp4"):

            mp4_files.append(os.path.join(directory, file))

            video_basename.append(os.path.splitext(file)[0])

    return mp4_files, video_basename

In [ ]:
# model = YOLO("yolov8n.pt")
# result = model.track("/content/videos/002.mp4", conf = 0.3, iou = 0.5, save = True, name='off_test_result')


# Example usage:
zip_file_path = './videos.zip'

extract_to_directory = './'

def unzip_file(zip_file_path, extract_to):
    """
    Unzip a file to a specified directory.

    Parameters:
        zip_file_path (str): Path to the zip file.
        extract_to (str): Directory where the contents of the zip file will be extracted.
    """
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

# Make sure the extraction directory exists
if not os.path.exists(extract_to_directory):
    os.makedirs(extract_to_directory)

# Unzip the file
unzip_file(zip_file_path, extract_to_directory)

In [ ]:
processed_video_folder_name = 'videos_new' # a directory folder to store processed video

if not os.path.exists(processed_video_folder_name):
   os.makedirs(processed_video_folder_name)

video_folder_name = 'data1' # directory that stores mp4 files

save_txt_path = './410985034.txt'

video_names, video_basenames = list_mp4_files(video_folder_name)


# Processed Videos shrink frames video by 2
for video, basename in zip(video_names, video_basenames):
    extract_frames(video, f'{processed_video_folder_name}/{basename}.mp4')

In [ ]:
def crop_images_from_video(video_path, crops):

    '''
    cropped detect bounding box
    ret : ROI images(list)
    '''

    cropped_images = []

    # Open the video
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return cropped_images

    frame_count = 0
    crop_index = 0
    crop_length = len(crops)

    while cap.isOpened() and crop_index < crop_length:
        ret, frame = cap.read()
        if not ret:
            break

        # Check if the current frame should be cropped
        while crop_index < crop_length and frame_count == crops[crop_index][2]:

            x1, y1, x2, y2 = [int(value) for value in crops[crop_index][3]]

            cropped_frame = frame[y1 : y2, x1 : x2]

            cropped_images.append(cropped_frame)

            crop_index += 1

        frame_count += 1

    cap.release()
    return cropped_images

In [ ]:
def fit_line_ransac(x, y, num_iterations=1000, threshold= 0.2):
    print(f"Number of masks: {len(x)}")
    assert(len(x) == len(y)), "length of input x, y is not equal."
    best_model = None
    best_inliers = []
    best_error = np.inf

    picked_size = max(min(len(x) // 2, 6), 1)

    for _ in range(num_iterations):
        # Randomly select 2 points
        indices = np.random.choice(len(x), picked_size, replace=False)

        x_sample = x[indices]
        y_sample = y[indices]

        # Fit a line to the selected points
        model = LinearRegression()
        model.fit(x_sample.reshape(-1, 1), y_sample)

        # Calculate distances from all points to the line
        distances = np.abs(model.predict(x.reshape(-1, 1)) - y)

        # Count inliers (points within threshold distance to the line)
        inliers = np.where(distances < threshold)[0]

        if(len(inliers)) > 0:
          # Calculate error (mean squared error of inliers)
          # error = mean_squared_error(y[inliers], model.predict(x[inliers].reshape(-1, 1)))

          error = sum(distances[inliers])

          # Update best model if this model has lower error and more inliers
          if len(inliers) > len(best_inliers) or (len(inliers) == len(best_inliers) and error < best_error):
              best_model = model
              best_inliers = inliers
              best_error = error

    return best_model, best_inliers

In [ ]:
def Extract_good_detections(result):
    '''
    ret : map
    '''
    # record good frames & detections

    appeared_id = {}
    for idx, r in enumerate(result):

        boxes = r.boxes  # Boxes object for bbox outputs

        # classes = r.cls
        probs = r.probs  # Class probabilities for classification outputs

        if boxes.cls.numel() != 0: # Prediction Not NULL

            for box in boxes:

              if box.id is not None:

                id = int(box.id[0])
                xyxy = box.xyxy[0]
                cls = int(box.cls[0])
                conf = float(box.conf)

                width, height = xyxy[2] - xyxy[0], xyxy[3] - xyxy[1]
                ratio = width / height

                value = {'conf': conf, 'cls': cls, 'idx' : idx, 'box' : xyxy}

                if 1.2 < ratio and ratio < 6 and width > 80 and height > 10:

                  if id in appeared_id:

                    if conf > appeared_id[id]['conf']:

                      appeared_id[id] = value
                  else:
                    appeared_id[id] = value

    return appeared_id

In [ ]:
def get_license(cropped_images):

    # List to store extracted text from each image
    extracted_texts = []

    # Iterate over each image in the list
    for img in cropped_images:

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        height, width = img.shape[0:2]

        # width, height = gray.size
        # # # histogram equalization
        # equ = cv2.equalizeHist(gray)

        # blurred = cv2.GaussianBlur(equ, (5, 5), 0)

        # cv2_imshow(blurred)

        _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

        cv2_imshow(binary)

        # Find contours in the binary mask
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        x_coord = []
        y_coord = []

        # # Loop over the contours
        for contour in contours:

            # Calculate the bounding rectangle for each contour
            x, y, w, h = cv2.boundingRect(contour)

            if ((w / width < 0.5) and (h / height < 0.8)):
              # cv2.rectangle(img, (x,y), (x+w, y+h), (0, 255, 0), 2)
              x_coord.append((x + (w //2)) / width)
              y_coord.append((y + (h //2)) / height)

        if(len(x_coord) == 0):
           # Use pytesseract to do OCR on the image
          result = reader.readtext(binary)

          for (bbox, text, prob) in result:

            if len(text) >= 3:
              extracted_texts.append(text)

        else:
          x_coord, y_coord = np.array(x_coord), np.array(y_coord)

          best_Model, best_inliers = fit_line_ransac(x_coord, y_coord, 50, 0.05)

          min_x, min_y, max_x, max_y = width, height, 0, 0

          for idx in best_inliers:

              contour = contours[idx]

              # Calculate the bounding rectangle for each contour
              x, y, w, h = cv2.boundingRect(contour)

              min_x = min(min_x, x)
              min_y = min(min_y, y)
              max_x = max(max_x, x + w)
              max_y = max(max_y, y + h)

          min_x = max(0, min_x)
          min_y = max(0, min_y)
          max_x = min(width, max_x)
          max_y = min(height, max_y)

          if len(best_inliers) < 2:
            license_plate = binary
          else:fAH
            license_plate = img[min_y:max_y, min_x:max_x]

          cv2_imshow(license_plate)

          clahe = cv2.createCLAHE()
          license_plate = clahe.apply(license_plate)

          # Use pytesseract to do OCR on the image
          result = reader.readtext(license_plate)

          for (bbox, text, prob) in result:

            if len(text) >= 4:
              extracted_texts.append(text)

        # Append the extracted text to the list

    return extracted_texts

In [ ]:
license_list = []

for basename in video_basenames:

  video_directory = f'{processed_video_folder_name}/{basename}.mp4'

  print('================================')
  print(f'reading video:{video_directory}')
  print('================================')

  # Obtain predicted results on processed Videos
  results = Best_Model.track( video_directory, conf = 0.3, iou = 0.5, name='test_result')

  appeared_id = Extract_good_detections(results)

  assert isinstance(appeared_id, dict), "'appeared_id' is not a dictionary (map)"

  ordered_frame = [(v['conf'], v['cls'], v['idx'], v['box']) for v in sorted(appeared_id.values(), key=lambda x: x['idx'])]

  print('================================')

  print(f'Get {len(ordered_frame)} Detections')
  print(ordered_frame)

  print('================================')

  # # Cropped images
  cropped_images = crop_images_from_video(video_directory, ordered_frame)

  extract_texts = get_license(cropped_images)

  print(f'License plate is: {extract_texts}')

  # cv2_imshow(cropped_images[0])
  license_list.append(extract_texts)

In [ ]:
# Check if the file exists and delete it if it does
if os.path.exists(save_txt_path):
        os.remove(save_txt_path)

for license, basename in zip(license_list, video_basenames):

   joined_license = ' '.join(license)

   with open(save_txt_path, 'a') as file:

        file.write(f'{basename} ')
        # Write the joined text to the file
        file.write(joined_license + '\n')

In [ ]:
from google.colab import files

# Download the file
files.download(save_txt_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>